In [134]:
# Import necessary modules
import pandas as pd

In [135]:
# Import the data

# Configure data Types for the DataFrame creation.  Integers are left out because NaNs generate an error
dtypes = {'ID': str, 
          'Device ID': str, 
          'Vehicle Type': 'category', 
          'Trip Duration': float, 
          'Trip Distance': float, 
          'Census Tract Start': str, 
          'Census Tract End': str}

# Configure ID as the index column.
index_cols = ['ID']

# Create a dataframe and print the head.
%time scooters = pd.read_csv('Shared_Micromobility_Vehicle_Trips.csv', dtype=dtypes, index_col=index_cols)
print(scooters.head())

# Save the size of the original dataframe
total_rows = scooters.shape[0]
print(total_rows)

CPU times: user 31.2 s, sys: 9.18 s, total: 40.4 s
Wall time: 44.3 s
                                                                 Device ID  \
ID                                                                           
6865731e-5e4b-4819-9435-4e62e09a652d  08c8c852-257c-49ad-a14f-727bfc79626d   
08dc0d9c-735e-4006-b9af-60011509e469  255716b5-4a9f-4efc-9967-7a65b3521bac   
92b80c20-89a4-4c7f-8b61-81a3570a218e  4d1e7bd0-75ce-401c-a802-8dd87122f802   
d9aef7c5-025d-4f2c-bafe-7801f89ef9dd  fdc38815-fe35-454b-bfb9-a63a2f2c205b   
baec3b6e-66f1-4043-bbd5-d54087f1c2de  007f1fed-0fe6-423f-8ca7-cd67834f20da   

                                     Vehicle Type  Trip Duration  \
ID                                                                 
6865731e-5e4b-4819-9435-4e62e09a652d          NaN            NaN   
08dc0d9c-735e-4006-b9af-60011509e469      bicycle     11491603.0   
92b80c20-89a4-4c7f-8b61-81a3570a218e      scooter          678.0   
d9aef7c5-025d-4f2c-bafe-7801f89ef9dd      sc

In [136]:
# Find which columns have na values
print(scooters.isna().any())

# Find how many rows are in the data.
rows_remaining = scooters.shape[0]
print(rows_remaining)

Device ID                   False
Vehicle Type                 True
Trip Duration                True
Trip Distance                True
Start Time                   True
End Time                     True
Modified Date               False
Month                        True
Hour                         True
Day of Week                  True
Council District (Start)     True
Council District (End)       True
Year                         True
Census Tract Start           True
Census Tract End             True
dtype: bool
6848951


In [137]:
'''Remove rows for which date time, Census Tract End or Census Tract Start are empty.
The entire row should be removed because prediction will not be possible on rows missing this information.
It also does not make sense to fill these columns, as every trip/row is a discrete event.'''
scooters = scooters.dropna(how='any', subset=['Start Time', 'End Time', 'Vehicle Type','Census Tract Start', 'Census Tract End'])

# Print the shape to see how many rows were removed.
na_rows_dropped = rows_remaining - scooters.shape[0]
rows_remaining = scooters.shape[0]
print(na_rows_dropped)
print(rows_remaining)

# Check if any Na values remain.
print(scooters.isna().any())

132
6848819
Device ID                   False
Vehicle Type                False
Trip Duration               False
Trip Distance               False
Start Time                  False
End Time                    False
Modified Date               False
Month                       False
Hour                        False
Day of Week                 False
Council District (Start)    False
Council District (End)      False
Year                        False
Census Tract Start          False
Census Tract End            False
dtype: bool


In [138]:
# Now these columns can be converted to int (importing as int generated errors from the NaN values):
scooters['Day of Week'] = scooters['Day of Week'].astype('int8')
scooters['Month'] = scooters['Month'].astype('int8')
scooters['Hour'] = scooters['Hour'].astype('int8')
scooters['Year'] = scooters['Year'].astype('int16')

In [139]:
%%time 

# Convert date columns to datetimes. 
# Do this instead of parse_dates in read_csv to take advantage of any reduced number of rows.
# This also allows format to be specified, which runs faster than making pandas guess the format.
scooters['Start Time'] = pd.to_datetime(scooters['Start Time'], format='%m/%d/%Y %I:%M:%S %p')
scooters['End Time'] = pd.to_datetime(scooters['End Time'], format='%m/%d/%Y %I:%M:%S %p')
scooters['Modified Date'] = pd.to_datetime(scooters['Modified Date'], format='%m/%d/%Y %I:%M:%S %p')

CPU times: user 1min 22s, sys: 888 ms, total: 1min 23s
Wall time: 1min 24s


In [140]:
# Print the data types to show each column is the correct type.
print(scooters.dtypes)

Device ID                           object
Vehicle Type                      category
Trip Duration                      float64
Trip Distance                      float64
Start Time                  datetime64[ns]
End Time                    datetime64[ns]
Modified Date               datetime64[ns]
Month                                 int8
Hour                                  int8
Day of Week                           int8
Council District (Start)           float64
Council District (End)             float64
Year                                 int16
Census Tract Start                  object
Census Tract End                    object
dtype: object


In [141]:
# Use describe to search for outliers
scooters.describe()

,Trip Duration,Trip Distance,Month,Hour,Day of Week,Council District (Start),Council District (End),Year
count,6.848819e+06,6.848819e+06,6.848819e+06,6.848819e+06,6.848819e+06,6.848819e+06,6.848819e+06,6.848819e+06
mean,6.683567e+02,2.567904e+04,6.365451e+00,1.467579e+01,3.249692e+00,7.724952e+00,7.653362e+00,2.018663e+03
std,1.904785e+03,6.602799e+06,3.144655e+00,5.063322e+00,2.091870e+00,2.607489e+00,2.660768e+00,4.725357e-01
min,-8.616100e+04,-1.672411e+07,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.018000e+03
25%,2.320000e+02,4.550000e+02,4.000000e+00,1.200000e+01,1.000000e+00,9.000000e+00,9.000000e+00,2.018000e+03
50%,4.050000e+02,9.980000e+02,6.000000e+00,1.500000e+01,3.000000e+00,9.000000e+00,9.000000e+00,2.019000e+03
75%,7.550000e+02,1.895000e+03,9.000000e+00,1.800000e+01,5.000000e+00,9.000000e+00,9.000000e+00,2.019000e+03
max,1.888695e+06,2.147484e+09,1.200000e+01,2.300000e+01,6.000000e+00,1.000000e+01,1.000000e+01,2.019000e+03


In [147]:
# Remove any rows that started or ended out of bounds
scooters = scooters[~scooters['Census Tract Start'].str.contains('OUT_OF_BOUNDS')]
scooters = scooters[~scooters['Census Tract End'].str.contains('OUT_OF_BOUNDS')]

# Get number of rows dropped
oob_rows_dropped = rows_remaining - scooters.shape[0]
rows_remaining = scooters.shape[0]
print(oob_rows_dropped)
print(rows_remaining)

55329
6793490


In [148]:
# Describe showed negative trip duration values. Remove these entries
scooters = scooters[scooters['Trip Duration'] > 0]
# Get number of rows dropped
neg_dur_rows_dropped = rows_remaining - scooters.shape[0]
rows_remaining = scooters.shape[0]
print('Negative duration dropped: ' + str(neg_dur_rows_dropped))
print(rows_remaining)

# Drop > 24 hours
scooters = scooters[scooters['Trip Duration'] < 86400]
high_dur_rows_dropped = rows_remaining - scooters.shape[0]
rows_remaining = scooters.shape[0]
print('Over 24 hours dropped: ' + str(high_dur_rows_dropped))
print(rows_remaining)

# Describe showed negative trip distance values. Remove these entries and any entries over 500 miles.
scooters = scooters[scooters['Trip Distance'] > 0]

neg_dist_rows_dropped = rows_remaining - scooters.shape[0]
rows_remaining = scooters.shape[0]
print('Negative distance dropped: ' + str(neg_dist_rows_dropped))
print(rows_remaining)


scooters = scooters[scooters['Trip Distance'] < 804672]
high_dist_rows_dropped = rows_remaining - scooters.shape[0]
rows_remaining = scooters.shape[0]
print('Distance > 500 dropped: ' + str(high_dist_rows_dropped))
print(rows_remaining)

Negative duration dropped: 1406
6792084
Over 24 hours dropped: 144
6791940
Negative distance dropped: 515895
6276045
Distance > 500 dropped: 510
6275535


In [150]:
# print number of rows dropped
(total_rows - rows_remaining)/total_rows

0.08372318622224045